In [ ]:
import re
import numpy
def date_sorter():
    
    # 0 - 124
    df_new_1 = df.str.findall(r'\d{1,2}?[-/](?:[0][0-9]|[0-9]|[1-2][0-9]|[3][0-1])[-/](?:\d{4}|\d{2})')

    # 125 - 342 
    df_new_2 = df.str.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?:\d{1,2},? )?\d{4}')

    # 343 - 454
    df_new_3 = df.str.findall(r'\d{,2}[/-]\d{4}')

    # 455 - 499
    df_new_4 = df.str.findall(r'\d{4}')

    def df_subset(df,row_start, row_end):
        x = pd.DataFrame(df.str[0]).loc[row_start:row_end,:] 
        return x

    df_new_1 = df_subset(df_new_1,0,124)
    df_new_2 = df_subset(df_new_2,125,342)
    df_new_3 = df_subset(df_new_3,343,454)
    df_new_4 = df_subset(df_new_4,455,499)

    df_new = pd.concat([df_new_1,df_new_2,df_new_3,df_new_4])
    # convert df to series
    df_new = df_new[0] 

    # replace spaces and dashes with / 
    df_new = df_new.str.replace(r'[-.,/ ]+','/')

    month_dict = {'Jan':'01','Feb':'02', 'Mar':'03', 'Apr': '04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov': '11', 'Dec':'12'}

    for i, value in enumerate(df_new):
        for month, initial in month_dict.items():
            if value.lower().count(month.lower()) > 0:
                df_new[i] = str(re.findall(month+r'[a-z/]*',value)[0]+re.sub(month+r'[a-z/]+','',value))

    # Month Dictionary 
    month_dict = {'Jan':'01','Feb':'02', 'Mar':'03', 'Apr': '04', 'May':'05', 'Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov': '11', 'Dec':'12'}

    for i, value in enumerate(df_new): 
        for month, initial in month_dict.items():
            if month.lower() in value.lower():
                df_new[i] = re.sub(month+r'[a-z]*',initial,value)

    for i, value in enumerate(df_new):
        dash_count = value.count('/')
        if dash_count == 1:
            df_new[i] = re.sub(r'(/)', r'/1/', value)
        elif dash_count == 0:
            df_new[i] = '1/1/'+value

    df_new = pd.to_datetime(df_new)

    df_new = pd.DataFrame(df_new)
    df_new['index'] = df_new.index

    df_final = df_new.sort_values(by = [0])['index'].reset_index()['index']    
    df_final = df_final.astype(numpy.int32)
    return df_final